# NOTE

- Make a copy of this template before you start editing, and exported the file as a ipynb where you are done.

- Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Runtime$\rightarrow$Factory reset runtime) and then **run all cells** (in the menubar, select Runtime$\rightarrow$Run all).

- Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE". Fill out your name below in the `NAME` variable, leave the `COLLABORATORS` variable empty.

In [27]:
NAME = "Ritesh Reddy"
COLLABORATORS = ""

---

Setup testing modules.

In [28]:
! pip install nose

In [29]:
import nose.tools as nt


def assert_dict_almost_equal(dict_1, dict_2):
    nt.assert_equal(set(dict_1.keys()), set(dict_2.keys()))
    for k in dict_1.keys():
        for v_1, v_2 in zip(dict_1[k], dict_2[k]):
            nt.assert_almost_equal(v_1, v_2)

# Multiple Sequence Alignment

We saw in lecture that a multiple sequence alignment M between $k$ strings $\mathbf{v_1, ..., v_k}$ is a $k \times q$ matrix, where $q=\{\max\{|\mathbf{v_i}|:i\in [k]\},...,\Sigma_{i=1}^{k}|\mathbf{v_i}|\}$ such that the $i^{th}$ row contains the characters of $\mathbf{v_i}$ in order with spaces '-' interspersed and no column cntains $k$ spaces. 

In this assignment, we will implement progressive multiple sequence alignment using the greedy algorithm. We will need to implement several parts:
* Computing the profile representation of a multiple sequence alignment.
* Aligning a single sequence to a profile
* Aligning a profile to another profile
* The greedy progressive alignment algoirthm

## Part 1: Compute a profile - 10 points

Given a multiple sequence alignment, we must compute its profile representation. Recall that a profile for a multiple sequence alignment of length $l$ in an alphabet $\Sigma$ is a $(|\Sigma| + 1) \times l$ matrix, where $p_{i,j}$ is the frequency of the $i^{th}$ letter in the $j^{th}$ position of the alignment.

Complete the `compute_profile` function defined below which takes a multiple sequence alignment and an alphabet and returns its profile representation.

In [30]:
def compute_profile(alignment, alphabet):
    """
    Given an alphabet an a multiple sequence alignment in that alphabet,
    computes and returns its profile representation
    
    :param: alignment is a list of lists of characters in the alphabet
    :param: alphabet is a list of characters in the alphabet from which the strings are
            constructed
    :return: a dictionary where dict[x][i] is the frequency of the character
             x in the i-th position of the alignment.
    """
    
    if not alignment:
        return {}

    n = len(alignment)
    l = len(alignment[0])
    
    profile = {}
    for i in alphabet:
      profile[i] = []

    alignments = []
    for i in range(l):
      alignment_string = ''
      for j in range(n):
        alignment_string += alignment[j][i]
      alignments.append(alignment_string)

    
    for i in range(len(alignments)):
      for key in profile.keys():
        profile[key].append( float( alignments[i].count(key) ) / float( len(alignments[i]) ) )
    # YOUR CODE HERE
    # raise NotImplementedError()
    
    return profile

alphabet = ['A', 'C', 'G', 'T', '-']
test2_aln = ['-T', 'GT', 'G-','AT', 'GC']
test2_soln = {'-': [0.2, 0.2], 'A': [0.2, 0.0], 'C': [0.0, 0.2], 'G': [0.6, 0.0],'T': [0.0, 0.6]}
assert_dict_almost_equal(compute_profile(test2_aln, alphabet),test2_soln)

In [31]:
### Student tests (You may add more)
alphabet = ['A', 'C', 'G', 'T', '-']

# Test case 1
test1_aln = ['A', 'T', 'C', 'G']
test1_soln = {'-': [0.0], 'A': [0.25], 'C': [0.25], 'G': [0.25], 'T': [0.25]}

# Test case 2
test2_aln = ['-T', 'GT', 'G-','AT', 'GC']
test2_soln = {'-': [0.2, 0.2], 'A': [0.2, 0.0], 'C': [0.0, 0.2], 'G': [0.6, 0.0],'T': [0.0, 0.6]}

assert_dict_almost_equal(compute_profile(test1_aln, alphabet),test1_soln)
assert_dict_almost_equal(compute_profile(test2_aln, alphabet),test2_soln)

In [32]:
# HIDDEN TEST CELL DO NOT MODIFY


In [33]:
# HIDDEN TEST CELL DO NOT MODIFY


In [34]:
# HIDDEN TEST CELL DO NOT MODIFY


In [35]:
# HIDDEN TEST CELL DO NOT MODIFY


## Part 2: Alignment with profiles - 10 points

In order to do progressive alignment, we need to be able to align a sequence with an existing multiple sequence alignment, as well as be able to align two multiple alignments. To do this, we represent the multiple alignment as a profile, and align the single sequence against the profile or align two profiles.

Recall from HW1 that we wrote functions to align two sequences. This function took a scoring function as an argument to aid in the comparison of various alignments. Similarly, we need a scoring function $\mathbf{\tau}$ that scores the alignment a character in our alphabet to a column of the profile.

Complete the function `compute_tau` which takes a profile, a scoring function, and an alphabet, and returns the scoring function tau. 

In [36]:
def compute_tau(profile, alphabet, delta):
    """
    Given a profile, an alphabet and a scoring function for that alphabet,
    returns the scoring function for aligning a character in the alphabet
    to a column in the profile
    
    :param: profile is the profile representation of the multiple sequence
            we are aligning against
    :param: alphabet is the alphabet of characters that compose our sequences
    :param: delta is the scoring function between characters in our alphabet
    
    :return: The scoring function tau such that tau[x][i] is the score for aligning
             character x with column i of the profile.
    """
    tau = {}
    for key, value in profile.items():
      value = [0 for i in range(len(value))]
      tau[key] = value

    for key,value in tau.items():
      for j in range(len(value)):
        for prof_key in profile.keys():
          tau[key][j] += profile[prof_key][j] * delta[key][prof_key]

    # YOUR CODE HERE
    # raise NotImplementedError()
    return tau

alphabet = ['A', 'C', 'G', 'T', '-']
delta = {}
for i in range(len(alphabet)):
    delta[alphabet[i]] = {k : v for (k,v) 
                          in zip(alphabet, [1 if alphabet[i] == alphabet[j]  else -1 
                                  for j in range(len(alphabet))]
                         )}
test2_aln = ['-T', 'GT', 'G-','AT', 'GC']
test2_profile = compute_profile(test2_aln, alphabet)
test2_soln = {'-': [-0.6, -0.6],
              'A': [-0.6, -1.0],
              'C': [-1.0, -0.6],
              'G': [0.2, -1.0],
              'T': [-1.0, 0.2]}
assert_dict_almost_equal(compute_tau(test2_profile, alphabet, delta),test2_soln)

In [37]:
### Student tests (You may edit these)
alphabet = ['A', 'C', 'G', 'T', '-']
delta = {}
for i in range(len(alphabet)):
    delta[alphabet[i]] = {k : v for (k,v) 
                          in zip(alphabet, [1 if alphabet[i] == alphabet[j]  else -1 
                                  for j in range(len(alphabet))]
                         )}

# Test case 1
test1_aln = ['A', 'T', 'C', 'G']
test1_profile = compute_profile(test1_aln, alphabet)
test1_soln = {'-': [-1.0], 'A': [-0.5], 'C': [-0.5], 'G': [-0.5], 'T': [-0.5]}

# Test case 2
test2_aln = ['-T', 'GT', 'G-','AT', 'GC']
test2_profile = compute_profile(test2_aln, alphabet)
test2_soln = {'-': [-0.6, -0.6],
              'A': [-0.6, -1.0],
              'C': [-1.0, -0.6],
              'G': [0.2, -1.0],
              'T': [-1.0, 0.2]}

assert_dict_almost_equal(compute_tau(test1_profile, alphabet, delta),test1_soln)
assert_dict_almost_equal(compute_tau(test2_profile, alphabet, delta),test2_soln)

In [38]:
# HIDDEN TEST CELL DO NOT MODIFY


In [39]:
# HIDDEN TEST CELL DO NOT MODIFY


In [40]:
# HIDDEN TEST CELL DO NOT MODIFY


In [41]:
# HIDDEN TEST CELL DO NOT MODIFY


Below is the function that aligns a sequence to a profile using the above functions.

`align_sequence_profile` takes a multiple sequence alignment, a new sequence, an alphabet, and a scoring function, and returns the progressively aligned multiple sequence alignment. ** This function is implemented for you. ** 

Just like in HW1, we will need to not only keep track of optimal alignment scores in a DP table,  but also keep track of the alignments themselves with back pointers.

**Note : You do not need to write any code for this function**

In [42]:
UP = (-1, 0)
LEFT = (0, -1)
TOPLEFT = (-1, -1)

def traceback(aln1, aln2, pointers):
    i = len(aln1[0])
    j = len(aln2[0])
    new_al1 = [list(v) for v in aln1]
    new_al2 = [list(w) for w in aln2]
    while True:
        di, dj = pointers[i][j]
        if (di, dj) == LEFT:
            for seq1 in new_al1:
                seq1.insert(i, '-')
        if (di, dj) == UP:
            for seq2 in new_al2:
                seq2.insert(j, '-')
        i, j = i + di, j + dj
        if (i <= 0 and j <= 0):
            break
    new_alignment = []
    for seq in new_al1:
        new_alignment.append(''.join(seq))
    for seq in new_al2:
        new_alignment.append(''.join(seq))
    return new_alignment


def align_sequence_profile(alignment, sequence, alphabet, delta):
    """
    This function aligns a sequence against a multiple sequence alignment
    
    :param: alignment is the multiple sequence alignment are aligning against.
            This is a list of list of characters
    :param: sequence is the new sequence we are aligning to the multiple alignment.
            This is a list of characters
    :param: alphabet is a list of characters that could compose the sequences in
            the alignments.
    :param: delta is the scoring function for aligning characters in our alphabet.
            delta[x][y] is the score for aligning the characters x and y.
    
    
    :return: a list of lists of characters in the alphabet, representing the 
             new multiple sequence alignment
    """
    # Base case when there is an empty multiple alignment
    if not alignment:
        return [sequence]
    
    M = [[0 for j in range(len(alignment[0])+1)] for i in range(len(alignment)+1)] 
    pointers = [[(0,0) for j in range(len(alignment[0])+1)] for i in range(len(alignment)+1)]
    score = None
    
    profile = compute_profile(alignment, alphabet)
    tau = compute_tao_sequence_profile(profile, alphabet, delta)

    for i in range(len(sequence)+1):
        for j in range(len(alignment[0])+1):
            if i == 0 and j == 0:   
                M[i][j] = 0
            elif i == 0:
                M[i][j] = M[i][j-1] + tau['-'][j-1]
                pointers[i][j] = LEFT
            elif j == 0:
                M[i][j] = M[i-1][j] + delta[sequence[i-1]]['-']
                pointers[i][j] = UP
            else:
                best_sub = max([(LEFT, M[i][j-1] + tau['-'][j-1]), 
                               (UP, M[i-1][j] + delta[v[i-1]]['-']), 
                               (TOPLEFT, M[i-1][j-1] + tau[sequence[i-1]][j-1])], key = lambda x: x[1])
                pointers[i][j] = best_sub[0]
                M[i][j] = best_sub[1]

    score = M[-1][-1]
    return score, traceback(sequence, alignment, pointers)

## Part 3: Align profile to profile - 20 points


Now using the given example, let's write the function `align_profile_profile` which takes two multiple sequence alignments, an alphabet and the scoring function for that alphabet, and returns their optimal alignment score as well as the optimal alignment. This function is a bit more complicated than aligning a sequence to a profile. This is because we need 4 separate scoring functions.

Given a profile $p$ for the first alignment, and a profile $q$ representing the second, and alphabet $\Sigma$:

* $\delta(x,y)$ gives us the score for aligning character $x$ and character $y$.
* $\tau_1(x,i)$ gives us the score for aligning character $x$ at position $i$ in the first profile $p$.
* $\tau_2(y,j)$ gives us the score for aligning character $y$ at position $j$ in the second profile $q$.
* $\sigma(i,j)$ gives us the score for aligning the $i^{th}$ column of $p$ with the $j^{th}$ column of $q$.

The scoring function $\sigma$ can be computed as follows $$\sigma(i,j) = \underset{x\in\Sigma}{\sum}\underset{y\in\Sigma}{\sum}p(x,i)q(y,j)\delta(x,y)$$

Once we have $\sigma$, we can use it to come up with a recurrence for aligning two profiles. If $S[i,j]$ is the score for first $i$ columns of profile $p$ with the first $j$ columns of profile $j$, then we have:

$$S[i,j] = max \begin{cases}
0 & \text{$i=0$ and $j=0$}\\
S[i-1,j] + \tau_1(-,i),& i \gt 0\\
S[i,j-1] + \tau_2(-,j),& j \gt 0\\
S[i-1,j-1] + \sigma(i,j),& \text{$i\gt0$ and $j\gt0$}\\
\end{cases}$$

Given the recurrence, complete the following parts:
* Finish the `compute_sigma` function which computes the scoring function for aligning columns of profile.
* Complete the `align_profile_profile` function below using the given recurrence.

For tie breaking in `align_profile_profile`, please follow the priority defined in `align_sequence_profile` (the given function): (LEFT, UP, TOPLEFT).

In [43]:
import numpy as np
def compute_sigma(p,q,alphabet, delta):
    """
    :param: p is the profile for the first multiple alignment
    :param: q is the profile for the second multiple alignment
    :param: alphabet is the list of all characters in our sequences
    :param: delta is the scoring function for aligning characters in our alphabet
    
    :returns: a list of lists sigma such that sigma[i][j] is the score for aligning column
              i of p with column j of q
    """
    sigma = []
    size_p = len(p[alphabet[0]])
    size_q = len(q[alphabet[0]])
    sigma = np.zeros((size_p, size_q))
    
    for i in range(size_p):
      for j in range(size_q):
        for x in p.keys():
          for y in q.keys():
            sigma[i,j] += p[x][i]*q[y][j]*delta[x][y]

    sigma = np.ndarray.tolist(sigma)
    # YOUR CODE HERE
    # raise NotImplementedError()
    return sigma

alphabet = ['A', 'C', 'G', 'T', '-']
delta = {}
for i in range(len(alphabet)):
    delta[alphabet[i]] = {k : v for (k,v) 
                          in zip(alphabet, [1 if alphabet[i] == alphabet[j]  else -1 
                                  for j in range(len(alphabet))]
                         )}

# Test case 1
test1_aln1 = ['ATG', 'TAC', 'C-C', 'G-T']
test1_aln2 = ['G-A', 'T-C']
test1_profile1 = compute_profile(test1_aln1, alphabet)
test1_profile2 = compute_profile(test1_aln2, alphabet)
test1_soln = [[-0.5, -1.0, -0.5], [-0.75, 0.0, -0.75], [-0.5, -1.0, -0.5]]

nt.assert_almost_equal(compute_sigma(test1_profile1, test1_profile2, alphabet, delta),test1_soln)

In [44]:
### Student tests (You may edit these)
alphabet = ['A', 'C', 'G', 'T', '-']
delta = {}
for i in range(len(alphabet)):
    delta[alphabet[i]] = {k : v for (k,v) 
                          in zip(alphabet, [1 if alphabet[i] == alphabet[j]  else -1 
                                  for j in range(len(alphabet))]
                         )}

# Test case 1
test1_aln1 = ['ATG', 'TAC', 'C-C', 'G-T']
test1_aln2 = ['G-A', 'T-C']
test1_profile1 = compute_profile(test1_aln1, alphabet)
test1_profile2 = compute_profile(test1_aln2, alphabet)
test1_soln = [[-0.5, -1.0, -0.5], [-0.75, 0.0, -0.75], [-0.5, -1.0, -0.5]]

nt.assert_almost_equal(compute_sigma(test1_profile1, test1_profile2, alphabet, delta),test1_soln)

In [45]:
# HIDDEN TEST CELL DO NOT MODIFY


In [51]:
UP = (-1, 0)
LEFT = (0, -1)
TOPLEFT = (-1, -1)

def traceback(aln1, aln2, pointers):
    i = len(aln1[0])
    j = len(aln2[0])
    new_al1 = [list(v) for v in aln1]
    new_al2 = [list(w) for w in aln2]
    while True:
        di, dj = pointers[i][j]
        if (di, dj) == LEFT:
            for seq1 in new_al1:
                seq1.insert(i, '-')
        if (di, dj) == UP:
            for seq2 in new_al2:
                seq2.insert(j, '-')
        i, j = i + di, j + dj
        if (i <= 0 and j <= 0):
            break
    new_alignment = []
    for seq in new_al1:
        new_alignment.append(''.join(seq))
    for seq in new_al2:
        new_alignment.append(''.join(seq))
    return new_alignment

def align_profile_profile(aln1, aln2, alphabet, delta):
    """
    :param: aln1 is a list of lists representing the first multiple alignment
    :param: aln2 is a list of lists representing the second multiple alignment
    :param: alphabet is the alphabet from which the sequences are derived
    :param: delta is a scoring function. delta(x,y) gives us the score for aligning 
            character x with character y in our alphabet
            
    :returns: the optimal score and the optimal multiple alignment for the two input alignments.
    """
    # Base case when there is an empty multiple alignment
    if not aln1 and not aln2:
        return []
    elif not aln2:
        return aln1
    elif not aln1:
        return aln2
    
    S = [[0 for j in range(len(aln2[0])+1)] for i in range(len(aln1[0])+1)] 
    pointers = [[(0,0) for j in range(len(aln2[0])+1)] for i in range(len(aln1[0])+1)]
    score = None
    
    # Compute profiles and scoring functions here
    p = compute_profile(aln1, alphabet)
    q = compute_profile(aln2, alphabet)
    tau1 = compute_tau(p, alphabet, delta)
    tau2 = compute_tau(q, alphabet, delta)
    sigma = compute_sigma(p,q,alphabet, delta)

    for i in range(len(aln1[0])+1):
        for j in range(len(aln2[0])+1):
            if i == 0 and j == 0:   
                S[i][j] = 0
            elif i == 0:
                S[i][j] = S[i][j-1] + tau2['-'][j-1]
                pointers[i][j] = LEFT
            elif j == 0:
                S[i][j] = S[i-1][j] + tau1['-'][i-1]
                pointers[i][j] = UP
            else:
                best_sub = max([(LEFT, S[i][j-1] + tau2['-'][j-1]), 
                               (UP, S[i-1][j] + tau1['-'][i-1]), 
                               (TOPLEFT, S[i-1][j-1] + sigma[i-1][j-1])], key = lambda x: x[1])
                pointers[i][j] = best_sub[0]
                S[i][j] = best_sub[1]

    # YOUR CODE HERE
    # raise NotImplementedError()
    score = S[-1][-1]
    return score, traceback(aln1, aln2, pointers)

In [52]:
### Student tests (You may edit these)
alphabet = ['A', 'C', 'G', 'T', '-']
delta = {}
for i in range(len(alphabet)):
    delta[alphabet[i]] = {k : v for (k,v) 
                          in zip(alphabet, [1 if alphabet[i] == alphabet[j]  else -1 
                                  for j in range(len(alphabet))]
                         )}

# Test cases
nt.assert_almost_equal(align_profile_profile(['GTAGTAC'], ['TGTCAGC'], alphabet, delta),(1.0, ['GTAGT-A-C', '-T-GTCAGC']))

In [48]:
# HIDDEN TEST CELL DO NOT MODIFY


In [49]:
# HIDDEN TEST CELL DO NOT MODIFY


# Part 4: Greedy Progressive Alignment - 10 points

Finally, we will put all the pieces together into the function `greedy_progressive_align`. This function will take a list of sequences, an alphabet, and a scoring function $\delta$ which scores the alignment for characters in our alphabet.

We now state two important facts:
* A sequence is simply a multiple sequence alignment of length 1
* A sequence can also be represented as a profile. What is the implication of this?

Let's go over the algorithm in a little more detail.

Given a labelled or indexed list of sequences, an alphabet and a scoring function $\delta$, 

1. First compute the score of pairwise alignment of all alignments in your list of sequences
2. Now pick the maximum scoring alignment of alignments and combine it
3. Repeat steps 1 and 2 until there is only 1 alignment in your input list

Some of the function has been completed for you. You only need compute pairwise alignment scores, and keep track of the best score.

In [67]:
def greedy_progressive_align(alignments, alphabet, delta):
    """
    :param: alignments is a list of list of strings representing the sequences to be aligned
            Note: This is because we need to represent our single sequences as multiple alignments
            ,and multiple alignments are lists of strings
    :param: alphabet is the alphabet from which the sequences are derived
    :param: delta is a scoring function. delta(x,y) gives us the score for aligning 
            character x with character y in our alphabet
            
    :returns: the greedy optimal multiple sequence alignment for a given set of sequences, and the score for that alignment
    """
    
    
    while True: 
        # Base case (When to exit the loop?)
        # YOUR CODE HERE
        if(len(alignments) == 1):
          return best_score, best_alignment
        # raise NotImplementedError()
        
        # Data structures for this iteration
        best_score = -float("inf")
        best_alignment = None
        best_m = -1
        best_n = -1

        # Compute pairwise distances 
        for m in range(len(alignments)):
            for n in range(m):
              if m != n:
                profile_score = align_profile_profile(alignments[m], alignments[n], alphabet, delta)[0]
                if profile_score >= best_score:
                  best_score = profile_score
                  best_alignment = align_profile_profile(alignments[m], alignments[n], alphabet, delta)[1]
                  best_m = m
                  best_n = n
                else:
                  continue
              else:
                continue

                # YOUR CODE HERE
                # raise NotImplementedError()

        # Populate the list of alignments to use for the next iteration
        next_alignments = [best_alignment]
        for i in range(len(alignments)):
            if i!=best_m and i!=best_n:
                next_alignments.append(alignments[i])
        alignments = next_alignments   

In [68]:
# Student test case (You may add more)

alphabet = ['A', 'C', 'G', 'T', '-']
delta = {}
for i in range(len(alphabet)):
    delta[alphabet[i]] = {k : v for (k,v) 
                          in zip(alphabet, [1 if alphabet[i] == alphabet[j]  else -1 
                                  for j in range(len(alphabet))]
                         )}
# Test case
test1_seqs = [['A'],['C'],['T'],['G']]
nt.assert_almost_equal(greedy_progressive_align(test1_seqs, alphabet, delta),
             (-1,['G', 'T', 'C', 'A']))

In [65]:
# HIDDEN TEST CELL DO NOT MODIFY


In [66]:
# HIDDEN TEST CELL DO NOT MODIFY
